In [1]:
!pip install numpy
!pip install sentence-transformers
!pip install faiss-cpu
!pip install keybert
!pip install nltk

In [2]:
import json
import re
from functools import lru_cache
from datetime import datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from sentence_transformers import SentenceTransformer
import faiss
from keybert import KeyBERT
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import SnowballStemmer

In [3]:
# Инициализация модели для преобразования текста в векторы
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Инициализация KeyBERT для извлечения ключевых слов
kw_model = KeyBERT()

# Инициализация стеммеров для русского и английского языков
stemmer_ru = SnowballStemmer('russian')
stemmer_en = SnowballStemmer('english')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# Загрузка данных из JSON-файлов
def load_data():
    """
    Загружает данные из JSON-файлов и преобразует их в структурированный формат.
    Возвращает три объекта:
    - job_data: словарь вакансий
    - specialists_data: список кандидатов
    - skill_names: словарь нормализованных навыков
    """
    # Загрузка навыков
    with open('skills.json', encoding='utf-8') as f:
        skills = json.load(f)

    # Загрузка вакансий
    with open('jobs.json', encoding='utf-8') as f:
        jobs = json.load(f)

    # Загрузка данных о кандидатах
    with open('specialists.json', encoding='utf-8') as f:
        specialists = json.load(f)

    # Создание словаря для навыков
    skill_names = {
        s['_id']['$oid']: {     # Используем уникальный ID навыка как ключ
            'ru': s['name']['ru'].lower(),
            'en': s['name']['en'].lower()
        }
        for s in skills
    }

    # Обработка вакансий
    job_data = {}    # Словарь для хранения обработанных вакансий
    for job in jobs:
        job_id = job['_id']['$oid']  # Получаем уникальный ID вакансии
        # Формируем объединенный текст требований вакансии
        job_req = ' '.join([
            f"{r['requirement'].get('ru', '')} {r['requirement'].get('en', '')}"
            for r in job['requirements']
        ])
        # Сохраняем структурированные данные вакансии
        job_data[job_id] = {
            'requirements': job_req,
            'tasks': [t['name']['ru'] for t in job['tasks']],
            'mandatory_skills': [
                (r['requirement'].get('ru', '').lower(), r['mandatory'])
                for r in job['requirements']
            ]
        }

    # Обработка данных о кандидатах
    specialists_data = []   # Список для хранения профилей кандидатов
    for spec in specialists:
        skills_list = [skill_names[s]['ru'] for s in spec.get('skills', []) if s in skill_names]
        skills_text = '; '.join(skills_list)   # Объединяем навыки в строку

        # Расчет опыта работы
        experience = {}
        for exp in spec['experience']:
            try:
                # Парсим дату начала работы
                start = datetime.fromisoformat(exp['start']['$date']).replace(tzinfo=None)
                # Парсим дату окончания (или текущую дату, если нет окончания)
                end = (
                    datetime.fromisoformat(exp['end']['$date']).replace(tzinfo=None)
                    if exp['end']
                    else datetime.now().replace(tzinfo=None)
                )
                months = calculate_experience(start, end)

                desc = exp.get('description', {}).get('ru', '') + ' ' + exp.get('description', {}).get('en', '')
                # Извлекаем навыки из описания опыта работы
                exp_skills = extract_skills(desc, skill_names)

                # Аккумулируем опыт по навыкам
                for skill in exp_skills:
                    experience[skill] = experience.get(skill, 0) + months
            except (KeyError, ValueError):
                # Пропускаем некорректные записи опыта
                continue

        specialists_data.append({
            'id': spec['_id']['$oid'],
            'name': spec['name']['ru'],   # Имя на русском
            'skills': skills_text,   # Строка с навыками
            'experience': experience   # Словарь опыта
        })

    return job_data, specialists_data, skill_names

In [5]:
# Нормализация навыков
def normalize_skill(skill: str) -> str:
    # Разделение на русские и английские части
    tokens = skill.split()
    stemmed = []
    for token in tokens:
        if re.search('[а-яА-Я]', token):
            stemmed.append(stemmer_ru.stem(token))
        else:
            stemmed.append(stemmer_en.stem(token))
    return ' '.join(stemmed)

In [6]:
# Извлечение ключевых навыков
def extract_skills(text: str, skill_db: dict) -> list:
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 2)) # Извлечение ключевых слов
    validated_skills = []
    for kw, _ in keywords:
        # Проверка, есть ли ключевое слово в skills.json
        for skill_id, skill_names in skill_db.items():
            if normalize_skill(kw.lower()) in normalize_skill(skill_names['ru']) or normalize_skill(kw.lower()) in normalize_skill(skill_names['en']):
                validated_skills.append(skill_names['ru'])
    return list(set(validated_skills))

In [7]:
# Расчет опыта работы
def calculate_experience(start: datetime, end: datetime) -> int:
    delta = relativedelta(end, start)
    return min(delta.years * 12 + delta.months, 600)

In [8]:
# Семантическое сравнение навыков
def calculate_similarity(skill1: str, skill2: str, threshold: float = 0.7) -> bool:
    @lru_cache(maxsize=10_000) # Кэширование результатов для ускорения работы
    def get_embedding(skill: str):
        return model.encode(skill) # Преобразование навыка в вектор

    emb1 = get_embedding(skill1)
    emb2 = get_embedding(skill2)
    return cosine_similarity([emb1], [emb2])[0][0] >= threshold # Сравнение сходства

In [9]:
# Расчет соответствия кандидата вакансии
def calculate_match(job, spec, skill_db):
    """Расчет соответствия"""
    try:
        job_skills = extract_skills(job['requirements'], skill_db) # Извлечение навыков из вакансии
        candidate_skills = extract_skills(spec['skills'], skill_db) # Извлечение навыков кандидата

        if not job_skills:
            return {'match_percent': 0, 'matched_skills': [], 'missing_skills': []}

        # Проверка обязательных навыков
        mandatory_skills = [skill for skill, is_mandatory in job['mandatory_skills'] if is_mandatory]
        if not all(skill in candidate_skills for skill in mandatory_skills):
            return {'match_percent': 0, 'matched_skills': [], 'missing_skills': job_skills}

        # Создаем общий словарь навыков
        all_skills = list(set(job_skills + candidate_skills))

        # Кодируем все навыки
        skill_embeddings = model.encode(all_skills)

        # Создаем матрицу схожести
        similarity_matrix = cosine_similarity(skill_embeddings)

        # Находим соответствия
        matched = []
        for j_skill in job_skills:
            idx = all_skills.index(j_skill)
            similarities = similarity_matrix[idx]

            # Находим лучшее совпадение среди навыков кандидата
            best_match = max([
                (similarities[all_skills.index(c_skill)], c_skill)    # Сходство и название навыка кандидата
                for c_skill in candidate_skills
            ], default=(0, None))    # Если совпадений нет, возвращаем (0, None)

            if best_match[0] >= 0.7:     # Если сходство превышает порог 0.7, считаем навык совпадающим
                matched.append(j_skill)  # Добавляем навык в список совпадающих

        # Возвращаем результат в виде словаря
        return {
            'match_percent': len(matched)/len(job_skills)*100,    # Процент совпадения навыков
            'matched_skills': matched,    # Список совпадающих навыков
            'missing_skills': list(set(job_skills) - set(matched))     # Список недостающих навыков
        }

    # Обработка исключений
    except Exception as e:
        print(f"Matching error: {str(e)}")     # Выводим сообщение об ошибке
        return {'match_percent': 0, 'matched_skills': [], 'missing_skills': job_skills}    # Возвращаем нулевое соответствие в случае ошибки

In [10]:
# Основной процесс
job_data, specialists_data, skill_db = load_data()

# Генерация эмбеддингов для кандидатов
def get_specialist_embedding(spec):
    # Объединяем навыки и опыт в один текст
    combined_text = spec['skills'] + ' ' + ' '.join(spec['experience'].keys())

    # Кодируем объединенный текст (аналогично вакансиям)
    return model.encode(combined_text)

# Подготовка индекса FAISS
def prepare_faiss_index(specialists):
    embeddings = model.encode([s['skills'] + ' ' + ' '.join(s['experience'].keys()) for s in specialists])
    embeddings = np.array(embeddings).astype('float32')
    faiss.normalize_L2(embeddings)

    dimension = model.get_sentence_embedding_dimension()  # Получаем размерность модели
    index = faiss.IndexFlatIP(dimension) # Создаем индекс FAISS
    index.add(embeddings)  # Добавляем векторы в индекс
    return index

# Основной процесс
index = prepare_faiss_index(specialists_data) # Подготовка индекса FAISS

In [11]:
# Поиск подходящих кандидатов для каждой вакансии
results = {}
for job_id, job in job_data.items():
    job_emb = model.encode(job['requirements'])  # Кодируем требования вакансии
    faiss.normalize_L2(job_emb.reshape(1, -1)) # Нормализуем вектор

    # Поиск ближайших кандидатов с использованием FAISS
    scores, indices = index.search(job_emb.reshape(1, -1).astype('float32'), 10)

    candidates = []
    for idx, score in zip(indices[0], scores[0]):
        if idx == -1:
            continue

        spec = specialists_data[idx]
        match_info = calculate_match(job, spec, skill_db) # Расчет соответствия

        # Комбинированный балл (70% от FAISS, 30% от соответствия навыков)
        combined_score = 0.7 * score + 0.3 * (match_info['match_percent'] / 100)

        candidates.append({
            'name': spec['name'],
            'faiss_score': float(score),
            'combined_score': combined_score,
            **match_info
        })

    results[job_id] = sorted(candidates, key=lambda x: -x['combined_score'])[:5] # Сортировка по комбинированному баллу

In [18]:
# Пример вывода
job_id_to_search = "6724e2726fa02b99fe3c3410"
report = {
    "job_id": job_id_to_search,
    "requirements": job_data[job_id_to_search]['requirements'],
    "candidates": results[job_id_to_search][:5]
}

# Вывод отчета в формате JSON
print(json.dumps(report, indent=4, ensure_ascii=False))

{
    "job_id": "6724e2726fa02b99fe3c3410",
    "requirements": "Стек: 1С-Битрикс: Enterprise (Битрикс24/БУС), PHP 7.4+ , MySQL 8, Memcache, Redis, git, VueJS; Stack: 1C-Bitrix: Enterprise (Bitrix24/BUS), PHP 7.4+ , MySQL 8, Memcache, Redis, git, VueJS; Имеет опыт программирования от 4 лет; He has programming experience of 4 years or more; Умеет оценивать сроки и сообщать о рисках, вовремя сигнализировать о проблеме; Able to assess deadlines and communicate risks, signalling a problem in a timely manner; Знает как реализовать требуемую функциональность любой сложности как на базе CMS Битрикс используя штатные средства и/или API, так и на чистом PHP; Knows how to implement the required functionality of any complexity both on the basis of CMS Bitrix using in-house tools and/or API, and on pure PHP; Знает, в каких случаях стандартные инструменты и технологии будут неэффективными; Knows when standard tools and techniques will be ineffective; Ищет неэффективные места в коде/архитектуре/тест